# Cloud Computing




### 1. Configuración del Nombre de la Instancia
![Instance Name](./images/instance-name.png)

*Asignar un nombre para identificar la instancia en el panel de control de AWS. Siga el formato `name-instance`*

### 2. Selección del Sistema Operativo
![OS Selection](./images/OS-selection.png)

*En este paso, se selecciona la imagen del sistema operativo para la instancia EC2. Seleccione `Ubuntu`*

### 3. Selección del Tipo de Instancia
![Instance Type Selection](./images/instance-type-selection.png)

*Se elige el tipo de instancia que se ajusta a las necesidades de cómputo y memoria. Seleccione `t2.micro`*

### 4. Creación del Par de Claves
![Key Pair Creation](./images/key-pair-creation.png)

*Se crea un par de claves nuevo o se elige uno existente para conectarse a la instancia de manera segura. Nombrela siguiendo el format `name-key`*

### 5. Configuración de la Red
![Network Settings](./images/network-settings.png)

*Click en `edit` y asignarle un nombre con el formato `name-security-group`*
*Personalización del grupo de seguridad para controlar el tráfico de red entrante y saliente de la instancia.*






Finalmente, vamos a hacer este proceso en la instancia de AWS.

- Encender la instancia asignada a cada uno
- Conectarse a la instancia. Note que debe cambiar el nombre de la `key.pm` y la dirección ip de su instancia. La opción `-o ServerAliveInterval=60` nos permite mantener la sesión viva.
    
```bash
ssh -i "key-cristian.pem" -o ServerAliveInterval=60 ubuntu@ec2-3-85-174-98.compute-1.amazonaws.com

```
- Crear ssh key en la instancia

```bash
ssh-keygen -t ed25519 -C "your_email@example.com"  
```

- Copiar dicha key y pegarla en github
```bash
cat ~/.ssh/id_xxxxx.pub
```

- Clonar repositorio

- Instalar Docker, ejecutando los siguientes comandos:

```bash
sudo apt-get update
sudo apt-get install ca-certificates curl gnupg
sudo install -m 0755 -d /etc/apt/keyrings
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /etc/apt/keyrings/docker.gpg
sudo chmod a+r /etc/apt/keyrings/docker.gpg
echo \
  "deb [arch="$(dpkg --print-architecture)" signed-by=/etc/apt/keyrings/docker.gpg] https://download.docker.com/linux/ubuntu \
  "$(. /etc/os-release && echo "$VERSION_CODENAME")" stable" | \
  sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt-get update
sudo apt-get install docker-ce docker-ce-cli containerd.io docker-buildx-plugin docker-compose-plugin
```

- Construir imagenes y correr contenedores con `sudo docker-compose`
- Configuración de Reglas de Ingreso al Grupo de Seguridad
*Después de crear el grupo de seguridad, es importante agregar una regla para permitir el tráfico entrante al puerto 8501, especialmente si se planea ejecutar una aplicación en Streamlit.*
    
    1. Ve a la configuración del grupo de seguridad de tu instancia.
    2. Haz clic en "Editar reglas de ingreso".
    3. Agrega una nueva regla con los siguientes parámetros:
       - **Tipo**: Personalizado TCP
       - **Puerto**: 8501
       - **Origen**: 0.0.0.0/0 (para permitir tráfico desde cualquier lugar) o una IP específica por razones de seguridad.
    4. Guarda los cambios.

- Probar con la dirección ip pública y los datos previamente expuestos.
- Finalmente, para apagar la instancia, ejecutar el siguiente comando
```bash
sudo shutdown -h now
```

# 🔧 Ampliar el espacio en disco de tu instancia EC2 a 16 GB

Vamos a aumentar el tamaño del volumen EBS para evitar errores de espacio al construir imágenes con Docker.

---

## 🟦 1. Abrir tu instancia y acceder al volumen rápidamente

1. Ve a **AWS Console → EC2 → Instances**.  
2. Haz clic en la instancia que estás usando en el curso.  
3. Baja a la sección **Storage**.  
4. En **Block devices**, haz clic en el **ID del volumen** (ej: `vol-0abc1234567def890`).  
   > Así evitas tener que buscar el volumen manualmente.

---

## 🟩 2. Modificar el volumen a 16 GB

1. En la pantalla del volumen, selecciona:  
   **Actions → Modify Volume**
2. Cambia **Size** a **16 GiB**.  
3. Haz clic en **Modify → Confirm**.

> No es necesario apagar la instancia. El cambio se aplica en caliente.

---

## 🟨 3. Expansión dentro de la instancia (SSH)

En tu terminal conectada a la EC2, ejecuta:

### 🔍 Verifica el tamaño actual del disco
```bash
lsblk
```

Probablemente verás algo así:

```
nvme0n1      16G
└─nvme0n1p1  6.8G
```

Esto significa que el volumen ya es de 16 GB, pero **la partición sigue sin usar el espacio extra**.

---

## 🟥 4. Expandir la partición y el sistema de archivos

Ejecuta:

```bash
sudo growpart /dev/nvme0n1 1
sudo resize2fs /dev/nvme0n1p1
```

Ambos comandos juntos actualizan la partición y luego el sistema de archivos para usar el nuevo tamaño.

---

## 🟢 5. Confirmar que ya tienes espacio adicional

```bash
df -h
```

Deberías ver algo parecido a:

```
Filesystem      Size  Used Avail Use%
/dev/root        16G   4G   12G   25%
```

🎉 ¡Listo! Ahora tu instancia tiene 16 GB disponibles y Docker podrá construir sin errores.

---